In [171]:
# Usual imports
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
# print(os.listdir("../input"))

# Plotly based imports for visualization
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 777.1 MB 13 kB/s  eta 0:00:013    |████▎                           | 103.5 MB 2.8 MB/s eta 0:03:57     |████████████▉                   | 312.1 MB 2.6 MB/s eta 0:03:01     |██████████████████              | 434.7 MB 3.0 MB/s eta 0:01:56
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [172]:
papers= pd.read_excel('ACM_output.xlsx')
papers.head()

,Title,type,total_downloads,total_citations,date,author_1,author_2,link,abstract
0,Abenteuer informatik: hands-on exhibits for le...,RESEARCH-ARTICLE,258,4,2012-11-01,Jens Gallenbacher,NaN,https://doi.org/10.1145/2481449.2481487,Computational thinking is one of the pillars o...
1,Reflections on outreach programs in CS classes...,RESEARCH-ARTICLE,510,11,2012-02-01,Renate Thies,Jan Vahrenhold,https://doi.org/10.1145/2157136.2157281,To provide a unified view of any scientific fi...
2,Internationalization of computer science educa...,RESEARCH-ARTICLE,341,6,2010-03-01,Sarah Douglas,Art Farley,https://doi.org/10.1145/1734263.1734404,Internationalization of computer science educa...
3,Exploring the K-12 computer science curriculum...,RESEARCH-ARTICLE,36,0,2020-10-01,Meize Guo,Anne Ottenbreit-Leftwich,https://doi.org/10.1145/3421590.3421594,In order to create early exposure and to guide...
4,A music context for teaching introductory comp...,RESEARCH-ARTICLE,346,18,2009-07-01,Ananya Misra,Douglas Blank,https://doi.org/10.1145/1562877.1562955,"We describe myro.chuck, a Python module for co..."


In [173]:
# Creating a spaCy object
nlp = spacy.load('en_core_web_lg')

spaCy also comes with a built-in named entity visualizer that lets you check your model's predictions in your browser. You can pass in one or more Doc objects and start a web server, export HTML files or view the visualization directly from a Jupyter Notebook.

# Named Entity Recognition

Named Entity Recognition is an information extraction task where named entities in unstructured sentences are located and classified in some pre-defined categories such as the person names, organizations, locations, medical codes, time expressions, quantities, monetary values, percentages, etc.

In [174]:
doc = nlp(papers["abstract"][4])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [175]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

# Lemmatization

It is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form. Words like "ran" and "running" are converted to "run" to avoid having words with similar meanings in our data.

In [176]:
review = str(" ".join([i.lemma_ for i in doc]))
print(review)

we describe myro.chuck , a Python module for control music synthesis , and its application to teach introductory computer science . the module be build within the Myro framework use the ChucK programming language , and be use in an introductory computer science course combine robot , graphic and music . the result support the value of music in engage student and broaden their view of computer science .


In [177]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

The sentence looks much different now that it is lemmatized.

# Parts of Speech tagging


This is the process of marking up a word in a text (corpus) as corresponding to a particular part of speech,[1] based on both its definition and its context—i.e., its relationship with adjacent and related words in a phrase, sentence, or paragraph. A simplified form of this is commonly taught to school-age children, in the identification of words as nouns, verbs, adjectives, adverbs, etc.

In [178]:
# POS tagging
for i in nlp(review):
    print(i,"=>",i.pos_)

we => PRON
describe => VERB
myro.chuck => NUM
, => PUNCT
a => DET
Python => PROPN
module => NOUN
for => ADP
control => NOUN
music => NOUN
synthesis => NOUN
, => PUNCT
and => CCONJ
its => PRON
application => NOUN
to => PART
teach => VERB
introductory => ADJ
computer => NOUN
science => NOUN
. => PUNCT
the => DET
module => NOUN
be => AUX
build => VERB
within => ADP
the => DET
Myro => PROPN
framework => NOUN
use => VERB
the => DET
ChucK => PROPN
programming => NOUN
language => NOUN
, => PUNCT
and => CCONJ
be => VERB
use => NOUN
in => ADP
an => DET
introductory => ADJ
computer => NOUN
science => NOUN
course => NOUN
combine => VERB
robot => NOUN
, => PUNCT
graphic => ADJ
and => CCONJ
music => NOUN
. => PUNCT
the => DET
result => NOUN
support => VERB
the => DET
value => NOUN
of => ADP
music => NOUN
in => ADP
engage => NOUN
student => NOUN
and => CCONJ
broaden => VERB
their => PRON
view => NOUN
of => ADP
computer => NOUN
science => NOUN
. => PUNCT


In [179]:
# Parser for reviews
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in nlp(sentence) ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [180]:
tqdm.pandas()
papers["processed_abstract"] = papers["abstract"].progress_apply(spacy_tokenizer)

100%|██████████| 1000/1000 [00:25<00:00, 38.72it/s]


In [181]:
papers[['abstract','processed_abstract']].head(6)

,abstract,processed_abstract
0,Computational thinking is one of the pillars o...,computational thinking pillar acm csta standar...
1,To provide a unified view of any scientific fi...,provide unified view scientific field outreach...
2,Internationalization of computer science educa...,internationalization computer science educatio...
3,In order to create early exposure and to guide...,order create early exposure guide talent compu...
4,"We describe myro.chuck, a Python module for co...",describe myro.chuck python module control musi...
5,This work presents an approach how student-cen...,work present approach student center computer ...


# What is topic-modelling?


In machine learning and natural language processing, a topic model is a type of statistical model for discovering the abstract "topics" that occur in a collection of documents. Topic modeling is a frequently used text-mining tool for discovery of hidden semantic structures in a text body. Intuitively, given that a document is about a particular topic, one would expect particular words to appear in the document more or less frequently: "dog" and "bone" will appear more often in documents about dogs, "cat" and "meow" will appear in documents about cats, and "the" and "is" will appear equally in both. A document typically concerns multiple topics in different proportions; thus, in a document that is 10% about cats and 90% about dogs, there would probably be about 9 times more dog words than cat words.

The "topics" produced by topic modeling techniques are clusters of similar words. A topic model captures this intuition in a mathematical framework, which allows examining a set of documents and discovering, based on the statistics of the words in each, what the topics might be and what each document's balance of topics is. It involves various techniques of dimensionality reduction(mostly non-linear) and unsupervised learning like LDA, SVD, autoencoders etc.

Source: Wikipedia

In [182]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

data_vectorized = vectorizer.fit_transform(papers["processed_abstract"])

In [183]:
NUM_TOPICS = 10

In [184]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [185]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [186]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [187]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [188]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('security', 46.42595079497582), ('creativity', 33.048101082229145), ('problem', 32.61181032842132), ('process', 28.104425561269178), ('standard', 25.25504722904441), ('pattern', 22.027958680363493), ('computation', 21.57233254207945), ('style', 21.15305820579124), ('creative', 19.91431994977456), ('work', 19.847371086491282)]
Topic 1:
[('interview', 8.198293754203467), ('belief', 7.074393057872125), ('germany', 4.854613117457328), ('america', 3.8821420634103707), ('north', 3.599359650455091), ('shift', 3.4087132165360274), ('semi', 3.1179569926104485), ('lesson', 2.932139990207952), ('structured', 2.5823827488824223), ('demographic', 2.571306841026267)]
Topic 2:
[('student', 1897.2332940345452), ('course', 884.665172928769), ('use', 502.9145314685856), ('paper', 448.3145065980926), ('study', 429.09815591050454), ('program', 428.2098987091338), ('research', 368.79952929090547), ('programming', 365.4092466744862), ('education', 350.5063068019454), ('school', 318.568

In [189]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('student', 11.840890250246167), ('study', 1.0271137309139309), ('experience', 0.8076708918147386), ('work', 0.5436471165728852), ('learning', 0.53923027079508), ('learn', 0.5204197591621834), ('result', 0.4893806649729388), ('increase', 0.4757047131186328), ('engagement', 0.4633665141937564), ('high', 0.45677929281592766)]
Topic 1:
[('research', 5.569323930050537), ('study', 2.6596828980389837), ('education', 2.4002200779633296), ('paper', 1.1067704269739989), ('result', 1.012015516134455), ('area', 0.7819224473739397), ('compute', 0.7281224392070988), ('field', 0.6534628267041728), ('present', 0.6497634883033222), ('information', 0.6451683553945011)]
Topic 2:
[('course', 8.050435986324642), ('teach', 1.154180927452895), ('major', 0.690285613696139), ('student', 0.6280056082323967), ('university', 0.5651063667384055), ('offer', 0.5310732409509887), ('topic', 0.45167490060922844), ('level', 0.40474473853742965), ('new', 0.374810180225627), ('include', 0.36559953640

In [190]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('student', 0.6341386082970336), ('course', 0.2839540189147645), ('use', 0.20494908030127174), ('programming', 0.14121829946731432), ('study', 0.13261625476532948), ('paper', 0.13142246636698374), ('school', 0.12602802714222885), ('program', 0.1202728951699064), ('computing', 0.11464498934890682), ('education', 0.11084735037309926)]
Topic 1:
[('computing', 0.2528720072910485), ('teacher', 0.23314666553259414), ('research', 0.21473177926547687), ('education', 0.20507425588252498), ('use', 0.18445902509277592), ('paper', 0.14612252031421746), ('program', 0.12011366346967914), ('design', 0.11096105122591589), ('school', 0.10915963974885261), ('curriculum', 0.09928150948408068)]
Topic 2:
[('course', 0.8557326925133909), ('teach', 0.12432246142332382), ('computing', 0.07623987792072151), ('cloud', 0.06772163155583101), ('major', 0.06434229151766033), ('data', 0.06261908488823766), ('offer', 0.05515904567941037), ('mobile', 0.0508414075210528), ('new', 0.0502603132161459

In [205]:
# Transforming an individual sentence
text = spacy_tokenizer("spaCy also comes with a built-in named entity visualizer that lets you check your model's predictions in your browser. You can pass in one or more Doc objects and start a web server, export HTML files or view the visualization directly from a Jupyter Notebook.")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.13271881 0.0071441  0.35806651 0.00714697 0.29826097 0.00714286
 0.00714291 0.16809015 0.00714336 0.00714337]


The index in the above list with the largest value represents the most dominant topic for the given review.

# How to interpret this graph?

1- Topics on the left while their respective keywords are on the right.

2- Larger topics are more frequent and closer the topics, more the similarity.

3- Selection of keywords is based on their frequency and discriminancy.

Hover over the topics on the left to get information about their keywords on the right.

In [192]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
2      32.420952   8.115230       1        1  60.633798
4       0.648955  71.707100       2        1  16.798247
7      71.019684   9.971006       3        1   6.594105
3       3.785981  33.185135       4        1   4.069599
8     -42.666500  -9.222070       5        1   3.347132
6      -3.621689 -43.526516       6        1   2.819046
9      -3.632402  -4.171708       7        1   2.770553
0     -36.584400  35.804413       8        1   2.502457
1      44.391380  50.615383       9        1   0.304061
5      40.235809 -31.591665      10        1   0.161003, topic_info=             Term        Freq       Total Category  logprob  loglift
1214  programming  488.000000  488.000000  Default  30.0000  30.0000
1331     research  509.000000  509.000000  Default  29.0000  29.0000
1211      program  501.000000  501.000000  Default  28.0000  28.0000
304     computing  571.000000  571.000000  Default  27.0000  27.0000
1621          use  889.000000  889.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
770     implement    0.065051  100.422241  Topic10  -7.4373  -0.9105
1491    structure    0.064955   90.740323  Topic10  -7.4388  -0.8106
498     education    0.065954  522.993631  Topic10  -7.4235  -2.5469
304     computing    0.065407  571.299104  Topic10  -7.4318  -2.6435
153          base    0.065276  378.486415  Topic10  -7.4338  -2.2338

[607 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
4         1  0.276200  abstract
4         2  0.069050  abstract
4         4  0.034525  abstract
4         7  0.034525  abstract
4         8  0.552400  abstract
...     ...       ...       ...
1681      1  0.859793      year
1681      2  0.109685      year
1681      5  0.024768      year
1683      3  0.881940      york
1686      2  0.887604   zealand

[1427 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 8, 4, 9, 7, 10, 1, 2, 6])

# Visualizing LSI(SVD) scatterplot

We will be visualizing our data for 2 topics to see similarity between keywords which is measured by distance with the markers using LSI model

In [193]:
svd_2d = TruncatedSVD(n_components=2)
data_2d = svd_2d.fit_transform(data_vectorized)

In [194]:
trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names(),
    hovertext = vectorizer.get_feature_names(),
    hoverinfo = 'text' 
)
data = [trace]
iplot(data, filename='scatter-mode')

# The text version of scatter plot looks messy but you can zoom it for great results

In [195]:
trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'text',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names()
)
data = [trace]
iplot(data, filename='text-scatter-mode')

# Let's see what happens when we use a spaCy based bigram tokenizer for topic modelling

In [196]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [197]:
bivectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, ngram_range=(1,2))
bigram_vectorized = bivectorizer.fit_transform(papers["abstract"])

# LDA for bigram data

In [198]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


# Topics for bigram model

In [199]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
[('students', 905.9217210750537), ('learning', 507.71175974453257), ('course', 440.1825616494798), ('programming', 350.1429053908217), ('paper', 320.3417916068465), ('based', 303.2098211762973), ('teaching', 287.58695717434324), ('software', 241.09734094423408), ('design', 233.33483983610617), ('using', 228.6247904716597)]
Topic 1:
[('unplugged', 20.547990162357078), ('science unplugged', 7.4412098662242), ('activities', 5.729580226173558), ('unplugged activities', 5.32407552532741), ('science lessons', 3.5382629748197116), ('way', 3.1655173495794067), ('outreach', 3.105361784497145), ('pattern', 2.890875016000991), ('uk', 2.577566038623979), ('exists', 2.556456142141354)]
Topic 2:
[('school', 248.46306000028468), ('teachers', 207.91590698341682), ('cs', 182.16755782922053), ('high', 156.8903118439065), ('computing', 137.89030201797158), ('students', 136.96620810482935), ('high school', 120.48340841463408), ('curriculum', 105.00153682781018), ('student', 99.03853

In [200]:
bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1  41.676651
3       0.648955  71.707100       2        1  20.278995
4      71.019684   9.971006       3        1  17.755596
2       3.785981  33.185135       4        1  11.958728
6     -42.666500  -9.222070       5        1   5.319828
7      -3.621689 -43.526516       6        1   1.223053
9      -3.632402  -4.171708       7        1   0.619416
5     -36.584400  35.804413       8        1   0.587466
1      44.391380  50.615383       9        1   0.339872
8      40.235809 -31.591665      10        1   0.240394, topic_info=             Term         Freq        Total Category  logprob  loglift
2147  programming   499.000000   499.000000  Default  30.0000  30.0000
2683     students  1648.000000  1648.000000  Default  29.0000  29.0000
647        course   626.000000   626.000000  Default  28.0000  28.0000
689            cs   792.000000   792.000000  Default  27.0000  27.0000
2596     software   250.000000   250.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2596     software     2.254094   250.571624  Topic10  -4.4305   1.3196
1003  engineering     1.086735   167.412426  Topic10  -5.1600   0.9934
2199       public     0.708148    59.975290  Topic10  -5.5883   1.5916
2164     programs     0.609598   204.496742  Topic10  -5.7382   0.2151
677      creation     0.427415    25.196563  Topic10  -6.0932   1.9539

[610 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5         1  0.033362       12
5         2  0.458732       12
5         3  0.016681       12
5         4  0.483754       12
5         8  0.008341       12
...     ...       ...      ...
3080      7  0.013530    years
3084     10  0.706129     york
3085      2  0.946787    young
3085      4  0.033814    young
3089      4  0.916502  zealand

[1441 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 5, 3, 7, 8, 10, 6, 2, 9])